In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
# Load the internet data and zip code shape files
# Internet data: https://data.cityofnewyork.us/City-Government/Broadband-Adoption-and-Infrastructure-by-Zip-Code/qz5f-yx82/data?no_mobile=true
# Zip code data: https://data.cityofnewyork.us/Health/Modified-Zip-Code-Tabulation-Areas-MODZCTA-/pri4-ifjk
# Zip code data is already in WGS84, so no conversion needed
internet = pd.read_csv(r"Broadband_Adoption_and_Infrastructure_by_Zip_Code_20231127.csv")
zipData = gpd.read_file(r"geo_export_fc721dfd-087f-41ed-b841-bec5c0e62515.shp")

In [ ]:
# Merge the internet data with the zip code shapes
# The modified zip code areas contain multiple internet locations
# Assign each internet area to its modified zip code 
zipData['modzcta'] = zipData['modzcta'].astype(int)
zipData = zipData.drop(index=177)
zipTab = []
for i in range(0, len(zipData['zcta'])):
    zipLab = zipData.iloc[i, 1]
    zipZCTA = zipData.iloc[i, 2]
    x = (zipLab + ', ' + zipZCTA)
    x = x.split(', ')
    x = [int(y) for y in x]
    zipTab.append(x)

# Assign by first match
internet['Zip Code'] = internet['Zip Code'].astype(int)
inds = []
for i in range(0, len(internet['Zip Code'])):
    x = internet.iloc[i, 1]
    for j in range(0, len(zipTab)):
        zipSet = zipTab[j]
        if x in zipSet:
            inds.append(j)
            break
        if j == (len(zipTab) - 1):
            inds.append(pd.NA)
    
# Perform the merge
internet = internet.assign(id = inds)
zipData = zipData.assign(id = range(0, zipData.shape[0]))
internet = internet.merge(
    zipData,
    on='id',
    how='left'
)

In [ ]:
# For internet observations with multiple zip codes
# Aggregate salient columns
# Just for demonstration - we can choose to do this in whatever way makes the most sense for the analysis
internet = internet.groupby(internet['id'], as_index=False).agg({
    'Home Broadband Adoption (Percentage of  Households)':'mean',
    'Mobile Broadband Adoption (Percentage of Households)':'mean',
    'No Internet Access (Percentage of Households)':'mean',
    'No Home Broadband Adoption (Percentage of  Households)':'mean',
    'No Mobile Broadband Adoption (Percentage of Households)':'mean',
    'Public Computer Center Count':'sum',
    'Public Wi-Fi Count':'sum',
    'modzcta':'first',
    'pop_est':'first',
    'geometry':'first'
})
internet = gpd.GeoDataFrame(internet)